In [217]:
# IMPORTS
import os
from collections import Counter
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET
import numpy as np
from scipy import sparse
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.gaussian_process import GaussianProcess

TRAIN_DIR = "train"
TEST_DIR = "test"

call_set = set([])

In [6]:
# UTILITIES
# these are the fifteen malware classes we're looking for
malware_classes = ["Agent", "AutoRun", "FraudLoad", "FraudPack", "Hupigon", "Krap",
           "Lipler", "Magania", "None", "Poison", "Swizzor", "Tdss",
           "VB", "Virut", "Zbot"]

# a function for writing predictions in the required format
def write_predictions(predictions, ids, outfile):
    """
    assumes len(predictions) == len(ids), and that predictions[i] is the
    index of the predicted class with the malware_classes list above for 
    the executable corresponding to ids[i].
    outfile will be overwritten
    """
    with open(outfile,"w+") as f:
        # write header
        f.write("Id,Prediction\n")
        for i, history_id in enumerate(ids):
            f.write("%s,%d\n" % (history_id, predictions[i]))

In [7]:
def add_to_set(tree):
    for el in tree.iter():
        call = el.tag
        call_set.add(call)

In [15]:
def create_data_matrix(start_index, end_index, direc="train"):
    X = None
    classes = []
    ids = [] 
    i = -1
    for datafile in os.listdir(direc):
        if datafile == '.DS_Store':
            continue

        i += 1
        if i < start_index:
            continue 
        if i >= end_index:
            break

        # extract id and true class (if available) from filename
        id_str, clazz = datafile.split('.')[:2]
        ids.append(id_str)
        # add target class if this is training data
        try:
            classes.append(malware_classes.index(clazz))

        except ValueError:
            # we should only fail to find the label in our list of malware classes
            # if this is test data, which always has an "X" label
            assert clazz == "X"
            classes.append(-1)

        # parse file as an xml document
        tree = ET.parse(os.path.join(direc,datafile))
        add_to_set(tree)
        this_row = call_feats(tree)
        if X is None:
            X = this_row 
        else:
            X = np.vstack((X, this_row))

    return X, np.array(classes), ids

In [153]:
good_calls = [
    'processes', 'query_value', 'read_section', 'read_section_names', 'read_value', 
        'recv_socket', 'remove_directory', 'revert_to_self', 'send_socket', 
        'set_file_attributes', 'set_file_time', 'set_system_time', 'set_thread_context', 
        'set_value', 'set_windows_hook', 'show_window', 'sleep', 'start_service', 'thread',
        'trimmed_bytes', 'unload_driver', 'vm_allocate', 'vm_mapviewofsection', 'vm_protect',
        'vm_read', 'vm_write', 'write_value', 'open_key', 'create_open_file', 'get_host_by_name', 
        'load_dll', 'create_mutex', 'load_image', 'create_file', 'get_file_attributes', 'open_process',
        'find_window', 'com_create_instance', 'destroy_window', 'impersonate_user', 'dump_line']

In [154]:
def call_feats(tree):
    
#     bigram_calls = []
#     for a_call in good_calls:
#         for b_call in good_calls:
#             bigram_calls.append(a_call + '_' + b_call)
            
#     combined_calls = good_calls + bigram_calls

    call_counter = {}
    last_call='start'
    for el in tree.iter():
        call = el.tag
        # Add specific call counts
        if call not in call_counter:
            call_counter[call] = 1
        else:
            call_counter[call] += 1
            
#         # Bigram Calls
#         combo = last_call + '_' + call
#         if combo not in call_counter:
#             call_counter[combo] = 1
#         #else:
#         #    call_counter[combo] += 1
#         last_call = call
            
    call_feat_array = np.zeros(len(good_calls))
    for i in range(len(good_calls)):
        call = good_calls[i]
        call_feat_array[i] = 0
        if call in call_counter:
            call_feat_array[i] = call_counter[call]

    return call_feat_array

## Import the data

In [191]:
# TRAINING DATA
X_train_all, t_train_all, train_all_ids = create_data_matrix(0, 3086, TRAIN_DIR)
X_train, X_valid, t_train, t_valid = train_test_split(X_train_all, t_train_all, test_size=0.20, random_state=37)

print 'Data matrix (training set):'
print X_train
print 'Classes (training set):'
print t_train

Data matrix (training set):
[[   1.    9.    0. ...,    1.    0.    0.]
 [   1.   14.    0. ...,    1.    0.    0.]
 [   1.  242.    0. ...,   42.    0.    0.]
 ..., 
 [   1.  242.    0. ...,   42.    0.    0.]
 [   1.    0.    0. ...,    0.    0.    0.]
 [   1.  534.    0. ...,    0.    3.   26.]]
Classes (training set):
[ 8  8  8 ...,  8  2 10]


In [158]:
# TESTING DATA
X_test_all, t_test_all, test_all_ids = create_data_matrix(0, 3724, TEST_DIR)

## Exploratory Data Analysis

## Test the models

In [201]:
# Try SVM
clf = svm.SVC()
clf.fit(X_train, t_train)
print "SVM"
print "Score of validation set: %f" % clf.score(X_valid, t_valid)

SVM
Score of validation set: 0.760518


### Random Forests

In [193]:
# Random Forest Classifier
print "Random Forests"
ns = [1,5,7,8,9,10,11,12,13,14,15,20,25,30,35,40,50,100,200]
for n in ns:
    rf = RandomForestClassifier(n_estimators=n)
    rf.fit(X_train, t_train)
    print "For n = %d, score was: %f" % (n, rf.score(X_valid, t_valid))

Random Forests
For n = 1, score was: 0.830097
For n = 5, score was: 0.865696
For n = 7, score was: 0.864078
For n = 8, score was: 0.875405
For n = 9, score was: 0.875405
For n = 10, score was: 0.870550
For n = 11, score was: 0.877023
For n = 12, score was: 0.888350
For n = 13, score was: 0.877023
For n = 14, score was: 0.877023
For n = 15, score was: 0.878641
For n = 20, score was: 0.883495
For n = 25, score was: 0.885113
For n = 30, score was: 0.883495
For n = 35, score was: 0.868932
For n = 40, score was: 0.877023
For n = 50, score was: 0.873786
For n = 100, score was: 0.875405
For n = 200, score was: 0.877023


In [268]:
rf = RandomForestClassifier(n_estimators=12, min_samples_split=3)
rf.fit(X_train, t_train)
print "Score was: %f" % (rf.score(X_valid, t_valid))

Score was: 0.870550


### Logisitic Regression

In [214]:
lr = LogisticRegression(penalty='l2',solver='newton-cg',max_iter=500)
lr.fit(X_train, t_train)
print "Score was: %f" % (lr.score(X_valid, t_valid))

Score was: 0.822006


In [215]:
lr = LogisticRegression(penalty='l1')
lr.fit(X_train, t_train)
print "Score was: %f" % (lr.score(X_valid, t_valid))

Score was: 0.818770


### Gaussian Process

In [218]:
gp = GaussianProcess()#theta0=0.1, thetaL=.001, thetaU=1.)
gp.fit(X_train, t_train)
print "Score was: %f" % (gp.score(X_valid, t_valid))

Exception: Multiple input features cannot have the same target value.

In [219]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, t_train)
print "Score was: %f" % (clf.score(X_valid, t_valid))

Score was: 0.283172


### KNC

In [203]:
# Try KNeighborsClassifier
ns = [1,2,3,4,5,6,7,8,10,20,25,30,50,100,200]
kneighborsscores = []
print "KNeighborsClassifier"
for n in ns:
    nn = KNeighborsClassifier(n_neighbors=n)
    nn.fit(X_train, t_train)
    score = nn.score(X_valid, t_valid)
    kneighborsscores.append(score)
    print "For n = %d, score was: %f" % (n, score)

KNeighborsClassifier
For n = 1, score was: 0.836570
For n = 2, score was: 0.838188
For n = 3, score was: 0.839806
For n = 4, score was: 0.844660
For n = 5, score was: 0.839806
For n = 6, score was: 0.834951
For n = 7, score was: 0.834951
For n = 8, score was: 0.834951
For n = 10, score was: 0.831715
For n = 20, score was: 0.813916
For n = 25, score was: 0.802589
For n = 30, score was: 0.789644
For n = 50, score was: 0.775081
For n = 100, score was: 0.745955
For n = 200, score was: 0.674757


In [265]:
kneighborsscores2 = []
nn = KNeighborsClassifier(n_neighbors=6, weights='distance')
nn.fit(X_train, t_train)
score = nn.score(X_valid, t_valid)
kneighborsscores2.append(score)
print "Score was: %f" % (score)

Score was: 0.864078


In [260]:
kneighborsscores2 = []
nn = KNeighborsClassifier(n_neighbors=6, weights='distance')
nn.fit(X_train, t_train)
score = nn.score(X_valid, t_valid)
kneighborsscores2.append(score)
print "Score was: %f" % (score)

Score was: 0.864078


In [264]:
from sklearn.neighbors import RadiusNeighborsClassifier
rnc = RadiusNeighborsClassifier(radius=6,outlier_label=8, p=2)
rnc.fit(X_train, t_train)
print "Score was: %f" % (rnc.score(X_valid, t_valid))

Score was: 0.745955


## Generate Predictions

In [96]:
# Generate predictions on test set for the best model

nn = KNeighborsClassifier(n_neighbors=6, weights='distance', p=1)
nn.fit(X_train_all, t_train_all)
test_predictions = nn.predict(X_test_all)

In [184]:
rf = RandomForestClassifier(n_estimators=12, min_samples_split=3)
rf.fit(X_train_all, t_train_all)
test_predictions = rf.predict(X_test_all)

In [185]:
def write_to_file(filename, ids, predictions):
    zips = zip(ids, predictions)
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(zips):
            f.write(str(p[0]) + "," + str(p[1]) + "\n")

In [186]:
write_to_file("prediction.csv", test_all_ids, test_predictions)

## Sandbox

In [48]:
# Run this to see what call_feats returns
direc = "train"
for idx, datafile in enumerate(os.listdir(direc)):
    if idx == 1:
        print datafile
        tree = ET.parse(os.path.join(direc,datafile))
        feats = call_feats(tree)
        
#         for el in tree.iter():
#             print el.tag
        print feats

00278ec420236020d6121dffe0cc20034422e7228.Lipler.xml
[  1.00000000e+00   2.00400000e+03   1.00000000e+00   0.00000000e+00
   5.10000000e+01   2.11000000e+02   1.00000000e+00   0.00000000e+00
   1.70000000e+01   2.00000000e+01   1.80000000e+01   0.00000000e+00
   0.00000000e+00   7.00000000e+01   2.80000000e+01   4.00000000e+00
   2.54000000e+02   0.00000000e+00   4.10000000e+01   2.05000000e+02
   0.00000000e+00   0.00000000e+00   0.00000000e+00   3.65000000e+02
   0.00000000e+00   0.00000000e+00   3.00000000e+00]
